In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2
import os

print(tf.__version__)

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from keras.initializers import RandomNormal

from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Conv1DTranspose,AveragePooling1D,GlobalMaxPool1D,GlobalAveragePooling1D
from tensorflow.keras.layers import Conv2D,MaxPool2D,Conv2DTranspose,AveragePooling2D,GlobalMaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv3D,MaxPooling3D,Conv3DTranspose,AveragePooling3D,GlobalMaxPool3D,GlobalAveragePooling3D
from tensorflow.keras.layers import Dense,Flatten,Dropout,Concatenate,Layer,BatchNormalization,Input,Add,Activation,Average

#from google.colab import drive


2.10.1


In [ ]:
#drive.mount('/content/drive')
#%cd drive/MyDrive/Edge/
#!ls

In [2]:
data_path = "H:\\download\\blender\\projects\\vdmTests\\outlineOutput"

In [3]:
train_path = data_path + "\\train_path\*"
edge_train_path = data_path + "\\edge_train_path\\*"
test_path = data_path + "\\test_path\\*"
edge_test_path = data_path + "\\edge_test_path\\*"

print(edge_test_path)

H:\download\blender\projects\vdmTests\outlineOutput\edge_test_path\*


In [4]:
def load_data(ipath, epath):
    images = sorted(glob(os.path.join(ipath)))
    edges = sorted(glob(os.path.join(epath)))
    return images, edges

In [5]:
images, edges = load_data(train_path, edge_train_path)
testimg, testedg = load_data(test_path, edge_test_path)

In [6]:
print(len(images), len(edges))

1001 1001


In [7]:
def read_image(path, H=512, W=512):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W,H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_edge_old(path, H=512, W=512):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    _, x = cv2.threshold(x,127,255,cv2.THRESH_BINARY)
    x = cv2.resize(x, (W,H))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    x = x.astype(np.float32)
    return x
def read_edge(path, H=512, W=512):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    #_, x = cv2.threshold(x,127,255,cv2.THRESH_BINARY)
    (_, x) = cv2.threshold(x, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #thresh = 127
    #x = cv2.threshold(x, thresh, 255, cv2.THRESH_BINARY)[1]
    x = cv2.resize(x, (W,H))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [8]:
def preprocess(x,y,H=512, W=512):
    
    def f(x,y):
        x = x.decode()
        y = y.decode()
        
        x = read_image(x)
        y = read_edge(y)
        return x, y
    
    images, edges = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([H, W, 3])
    edges.set_shape([H, W, 1])
    return images, edges

In [9]:
def tf_data(x,y):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.shuffle(buffer_size=10)
    data = data.map(preprocess)
    data = data.batch(1)
    data = data.prefetch(1)
    return data

In [10]:
train_data = tf_data(images, edges)
test_data = tf_data(testimg, testedg)

In [11]:
for x, y in train_data:
    print(x.shape, y.shape)
print(train_data)

(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 

(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 

(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 

(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 

(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 3) (1, 512, 512, 1)
(1, 512, 512, 

In [13]:
weight_init = tf.initializers.glorot_uniform()

l2 = regularizers.l2
w_decay=1e-3

glorot_normal = RandomNormal(stddev=0.01)

In [14]:

self_conv2D_3 = Conv2D(32 , kernel_size=(3,3),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_batchnormalization_24 = BatchNormalization()
self_conv2D_40 = Conv2D(64 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_batchnormalization_18 = BatchNormalization()
self_activation_11 = Activation(activation='relu')
self_conv2D_1 = Conv2D(128 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_16 = Conv2D(128 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_51 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_1 = BatchNormalization()
self_transpoze2D_9 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same')
self_conv2D_49 = Conv2D(128 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_8 = BatchNormalization()
self_activation_1 = Activation(activation='relu')
self_conv2D_17 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_maxpool2D_1 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_10 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same')
self_conv2D_28 = Conv2D(256 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_36 = Conv2D(256 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_activation_8 = Activation(activation='relu')
self_conv2D_47 = Conv2D(256 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_48 = Conv2D(512 , kernel_size=(1,1),strides=(2,2),padding = 'same')
self_conv2D_33 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_6 = BatchNormalization()
self_activation_9 = Activation(activation='relu')
self_conv2D_31 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_22 = BatchNormalization()
self_activation_6 = Activation(activation='relu')
self_conv2D_30 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_4 = BatchNormalization()
self_activation_7 = Activation(activation='relu')
self_conv2D_18 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_20 = BatchNormalization()
self_conv2D_38 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_maxpool2D_2 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_15 = Conv2DTranspose(16 , kernel_size=(4,4),strides=(2,2),padding = 'same')
self_conv2D_11 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_34 = Conv2D(512 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_50 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_2 = Conv2DTranspose(1 , kernel_size=(4,4),strides=(2,2),padding = 'same')
self_activation_10 = Activation(activation='relu')
self_conv2D_32 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_10 = BatchNormalization()
self_activation_5 = Activation(activation='relu')
self_conv2D_27 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_19 = BatchNormalization()
self_activation_24 = Activation(activation='relu')
self_conv2D_26 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_13 = BatchNormalization()
self_activation_16 = Activation(activation='relu')
self_conv2D_13 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_17 = BatchNormalization()
self_activation_22 = Activation(activation='relu')
self_conv2D_29 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_15 = BatchNormalization()
self_activation_23 = Activation(activation='relu')
self_conv2D_24 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_12 = BatchNormalization()
self_conv2D_44 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_maxpool2D_3 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_7 = Conv2DTranspose(16 , kernel_size=(8,8),strides=(2,2),padding = 'same')
self_conv2D_45 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_activation_15 = Activation(activation='relu')
self_conv2D_6 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_2 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_8 = Conv2DTranspose(16 , kernel_size=(8,8),strides=(2,2),padding = 'same')
self_conv2D_43 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_41 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_7 = BatchNormalization()
self_transpoze2D_14 = Conv2DTranspose(1 , kernel_size=(8,8),strides=(2,2),padding = 'same')
self_activation_17 = Activation(activation='relu')
self_conv2D_19 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_16 = BatchNormalization()
self_activation_13 = Activation(activation='relu')
self_conv2D_21 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_14 = BatchNormalization()
self_activation_14 = Activation(activation='relu')
self_conv2D_15 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_5 = BatchNormalization()
self_activation_18 = Activation(activation='relu')
self_conv2D_14 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_26 = BatchNormalization()
self_activation_19 = Activation(activation='relu')
self_conv2D_5 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_3 = BatchNormalization()
self_conv2D_8 = Conv2D(256 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_35 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_activation_20 = Activation(activation='relu')
self_transpoze2D_4 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_conv2D_22 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_12 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_11 = BatchNormalization()
self_transpoze2D_12 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_activation_21 = Activation(activation='relu')
self_conv2D_37 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_10 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_5 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_batchnormalization_21 = BatchNormalization()
self_conv2D_42 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_13 = Conv2DTranspose(1 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_activation_3 = Activation(activation='relu')
self_conv2D_20 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_9 = BatchNormalization()
self_activation_4 = Activation(activation='relu')
self_conv2D_23 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_2 = BatchNormalization()
self_activation_2 = Activation(activation='relu')
self_conv2D_7 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_23 = BatchNormalization()
self_activation_12 = Activation(activation='relu')
self_conv2D_25 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_25 = BatchNormalization()
self_conv2D_39 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_11 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_conv2D_4 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_3 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_conv2D_9 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_6 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same')
self_conv2D_46 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_1 = Conv2DTranspose(1 , kernel_size=(16,16),strides=(2,2),padding = 'same')
#self_block_cat = SingleConvBlock(
#    1,k_size=(1,1),stride=(1,1),
#    w_init=tf.constant_initializer(1/5))

self_last_conv = Conv2D(1, kernel_size=(1,1), strides=(1,1), padding = 'same', kernel_initializer=tf.constant_initializer(1/5))
self_last_batchnormalization = BatchNormalization()
self_last_activation = Activation(activation='relu')

#-------------------------------------

x = Input(shape=(512,512,3))
conv2D_3 = self_conv2D_3(x)
batchnormalization_24 = self_batchnormalization_24(conv2D_3)
conv2D_40 = self_conv2D_40(batchnormalization_24)
batchnormalization_18 = self_batchnormalization_18(conv2D_40)
activation_11 = self_activation_11(batchnormalization_18)
conv2D_1 = self_conv2D_1(activation_11)
conv2D_16 = self_conv2D_16(activation_11)
conv2D_51 = self_conv2D_51(activation_11)
batchnormalization_1 = self_batchnormalization_1(conv2D_1)
transpoze2D_9 = self_transpoze2D_9(conv2D_51)
conv2D_49 = self_conv2D_49(batchnormalization_1)
batchnormalization_8 = self_batchnormalization_8(conv2D_49)
activation_1 = self_activation_1(batchnormalization_8)
conv2D_17 = self_conv2D_17(activation_1)
maxpool2D_1 = self_maxpool2D_1(activation_1)
transpoze2D_10 = self_transpoze2D_10(conv2D_17)
add_1 = Add()([conv2D_16,maxpool2D_1])
conv2D_28 = self_conv2D_28(maxpool2D_1)
conv2D_36 = self_conv2D_36(maxpool2D_1)
activation_8 = self_activation_8(add_1)
conv2D_47 = self_conv2D_47(add_1)
conv2D_48 = self_conv2D_48(conv2D_36)
conv2D_33 = self_conv2D_33(activation_8)
batchnormalization_6 = self_batchnormalization_6(conv2D_33)
activation_9 = self_activation_9(batchnormalization_6)
conv2D_31 = self_conv2D_31(activation_9)
batchnormalization_22 = self_batchnormalization_22(conv2D_31)
average_5 = Average()([batchnormalization_22,conv2D_28])
activation_6 = self_activation_6(average_5)
conv2D_30 = self_conv2D_30(activation_6)
batchnormalization_4 = self_batchnormalization_4(conv2D_30)
activation_7 = self_activation_7(batchnormalization_4)
conv2D_18 = self_conv2D_18(activation_7)
batchnormalization_20 = self_batchnormalization_20(conv2D_18)
average_4 = Average()([batchnormalization_20,conv2D_28])
conv2D_38 = self_conv2D_38(average_4)
maxpool2D_2 = self_maxpool2D_2(average_4)
transpoze2D_15 = self_transpoze2D_15(conv2D_38)
add_2 = Add()([conv2D_47,maxpool2D_2])
add_4 = Add()([conv2D_36,maxpool2D_2])
conv2D_11 = self_conv2D_11(transpoze2D_15)
conv2D_34 = self_conv2D_34(add_2)
conv2D_50 = self_conv2D_50(add_4)
transpoze2D_2 = self_transpoze2D_2(conv2D_11)
activation_10 = self_activation_10(add_2)
conv2D_32 = self_conv2D_32(activation_10)
batchnormalization_10 = self_batchnormalization_10(conv2D_32)
activation_5 = self_activation_5(batchnormalization_10)
conv2D_27 = self_conv2D_27(activation_5)
batchnormalization_19 = self_batchnormalization_19(conv2D_27)
average_3 = Average()([batchnormalization_19,conv2D_50])
activation_24 = self_activation_24(average_3)
conv2D_26 = self_conv2D_26(activation_24)
batchnormalization_13 = self_batchnormalization_13(conv2D_26)
activation_16 = self_activation_16(batchnormalization_13)
conv2D_13 = self_conv2D_13(activation_16)
batchnormalization_17 = self_batchnormalization_17(conv2D_13)
average_2 = Average()([batchnormalization_17,conv2D_50])
activation_22 = self_activation_22(average_2)
conv2D_29 = self_conv2D_29(activation_22)
batchnormalization_15 = self_batchnormalization_15(conv2D_29)
activation_23 = self_activation_23(batchnormalization_15)
conv2D_24 = self_conv2D_24(activation_23)
batchnormalization_12 = self_batchnormalization_12(conv2D_24)
average_11 = Average()([batchnormalization_12,conv2D_50])
conv2D_44 = self_conv2D_44(average_11)
maxpool2D_3 = self_maxpool2D_3(average_11)
transpoze2D_7 = self_transpoze2D_7(conv2D_44)
add_3 = Add()([conv2D_34,maxpool2D_3])
add_5 = Add()([conv2D_48,maxpool2D_3])
conv2D_45 = self_conv2D_45(transpoze2D_7)
activation_15 = self_activation_15(add_3)
conv2D_6 = self_conv2D_6(add_3)
conv2D_2 = self_conv2D_2(add_5)
transpoze2D_8 = self_transpoze2D_8(conv2D_45)
conv2D_43 = self_conv2D_43(activation_15)
conv2D_41 = self_conv2D_41(transpoze2D_8)
batchnormalization_7 = self_batchnormalization_7(conv2D_43)
transpoze2D_14 = self_transpoze2D_14(conv2D_41)
activation_17 = self_activation_17(batchnormalization_7)
conv2D_19 = self_conv2D_19(activation_17)
batchnormalization_16 = self_batchnormalization_16(conv2D_19)
average_7 = Average()([batchnormalization_16,conv2D_2])
activation_13 = self_activation_13(average_7)
conv2D_21 = self_conv2D_21(activation_13)
batchnormalization_14 = self_batchnormalization_14(conv2D_21)
activation_14 = self_activation_14(batchnormalization_14)
conv2D_15 = self_conv2D_15(activation_14)
batchnormalization_5 = self_batchnormalization_5(conv2D_15)
average_6 = Average()([batchnormalization_5,conv2D_2])
activation_18 = self_activation_18(average_6)
conv2D_14 = self_conv2D_14(activation_18)
batchnormalization_26 = self_batchnormalization_26(conv2D_14)
activation_19 = self_activation_19(batchnormalization_26)
conv2D_5 = self_conv2D_5(activation_19)
batchnormalization_3 = self_batchnormalization_3(conv2D_5)
average_1 = Average()([batchnormalization_3,conv2D_2])
add_6 = Add()([average_1,conv2D_6])
conv2D_8 = self_conv2D_8(average_1)
conv2D_35 = self_conv2D_35(average_1)
activation_20 = self_activation_20(add_6)
transpoze2D_4 = self_transpoze2D_4(conv2D_35)
conv2D_22 = self_conv2D_22(activation_20)
conv2D_12 = self_conv2D_12(transpoze2D_4)
batchnormalization_11 = self_batchnormalization_11(conv2D_22)
transpoze2D_12 = self_transpoze2D_12(conv2D_12)
activation_21 = self_activation_21(batchnormalization_11)
conv2D_37 = self_conv2D_37(transpoze2D_12)
conv2D_10 = self_conv2D_10(activation_21)
transpoze2D_5 = self_transpoze2D_5(conv2D_37)
batchnormalization_21 = self_batchnormalization_21(conv2D_10)
conv2D_42 = self_conv2D_42(transpoze2D_5)
average_10 = Average()([batchnormalization_21,conv2D_8])
transpoze2D_13 = self_transpoze2D_13(conv2D_42)
activation_3 = self_activation_3(average_10)
conv2D_20 = self_conv2D_20(activation_3)
batchnormalization_9 = self_batchnormalization_9(conv2D_20)
activation_4 = self_activation_4(batchnormalization_9)
conv2D_23 = self_conv2D_23(activation_4)
batchnormalization_2 = self_batchnormalization_2(conv2D_23)
average_9 = Average()([batchnormalization_2,conv2D_8])
activation_2 = self_activation_2(average_9)
conv2D_7 = self_conv2D_7(activation_2)
batchnormalization_23 = self_batchnormalization_23(conv2D_7)
activation_12 = self_activation_12(batchnormalization_23)
conv2D_25 = self_conv2D_25(activation_12)
batchnormalization_25 = self_batchnormalization_25(conv2D_25)
average_8 = Average()([batchnormalization_25,conv2D_8])
conv2D_39 = self_conv2D_39(average_8)
transpoze2D_11 = self_transpoze2D_11(conv2D_39)
conv2D_4 = self_conv2D_4(transpoze2D_11)
transpoze2D_3 = self_transpoze2D_3(conv2D_4)
conv2D_9 = self_conv2D_9(transpoze2D_3)
transpoze2D_6 = self_transpoze2D_6(conv2D_9)
conv2D_46 = self_conv2D_46(transpoze2D_6)
transpoze2D_1 = self_transpoze2D_1(conv2D_46)
#tmp = [transpoze2D_1,transpoze2D_2,transpoze2D_9,transpoze2D_10,transpoze2D_13,transpoze2D_14]
tmp = [transpoze2D_9,transpoze2D_10,transpoze2D_2,transpoze2D_14,transpoze2D_13,transpoze2D_1]

concatenate_1 = tf.concat(tmp,3)
#print(f"concatenate_1 shape: {concatenate_1.shape}")
#return concatenate_1

#results = [transpoze2D_1,transpoze2D_2,transpoze2D_9,transpoze2D_10,transpoze2D_13,transpoze2D_14]
#block_cat = tf.concat(results, 3)  # BxHxWX6
#print(f"CONCATENATE_1 shape: {block_cat.shape}")
#block_cat = self_block_cat(block_cat)  # BxHxWX1
#results.append(block_cat)


last_conv =self_last_conv(concatenate_1)
last_batchnormalization = self_last_batchnormalization(last_conv)
last_activation = self_last_activation(last_batchnormalization)
#concatenate_1 = Concatenate()([transpoze2D_1,transpoze2D_3,transpoze2D_4,transpoze2D_7,transpoze2D_8,transpoze2D_10])




model = Model(inputs =x , outputs=last_activation)


H:\download\anaconda3_envs\generativeTensorflowCudaV2\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [15]:
#model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

                                                                                                  
 batch_normalization_7 (BatchNo  (None, 128, 128, 25  1024       ['conv2d_14[0][0]']              
 rmalization)                   6)                                                                
                                                                                                  
 average_1 (Average)            (None, 128, 128, 25  0           ['batch_normalization_7[0][0]',  
                                6)                                'conv2d_7[0][0]']               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 64, 64, 256)  33024       ['add[0][0]']                    
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 256)  0          ['average_1[0][0]']              
          

 batch_normalization_14 (BatchN  (None, 32, 32, 512)  2048       ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_13 (Activation)     (None, 32, 32, 512)  0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 conv2d_10 (Conv2D)             (None, 32, 32, 512)  131584      ['conv2d_8[0][0]']               
                                                                                                  
 conv2d_31 (Conv2D)             (None, 32, 32, 512)  2359808     ['activation_13[0][0]']          
                                                                                                  
 add_4 (Add)                    (None, 32, 32, 512)  0           ['conv2d_10[0][0]',              
          

 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 32, 32, 256)  0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_44 (Conv2D)             (None, 32, 32, 256)  590080      ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_23 (BatchN  (None, 32, 32, 256)  1024       ['conv2d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 average_9 (Average)            (None, 32, 32, 256)  0           ['batch_normalization_23[0][0]', 
          

                                                                                                  
 conv2d_30 (Conv2D)             (None, 256, 256, 1)  17          ['conv2d_transpose_5[0][0]']     
                                                                                                  
 conv2d_42 (Conv2D)             (None, 256, 256, 16  272         ['conv2d_transpose_9[0][0]']     
                                )                                                                 
                                                                                                  
 conv2d_50 (Conv2D)             (None, 256, 256, 1)  17          ['conv2d_transpose_13[0][0]']    
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 512, 512, 1)  5          ['conv2d_4[0][0]']               
 ose)                                                                                             
          

In [16]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.FalseNegatives(), 
             tf.keras.metrics.FalsePositives(), 
             tf.keras.metrics.TruePositives(), 
             tf.keras.metrics.TrueNegatives()])

In [ ]:
train_steps=len(train_data)
val_steps=len(test_data)
print(train_steps)
model.fit(train_data, validation_data=test_data, epochs=20, steps_per_epoch=train_steps, validation_steps=val_steps)

1001
Epoch 1/20
1001/1001 [==============================] - 599s 577ms/step - loss: 1.8640 - binary_accuracy: 0.4051 - false_negatives: 155371712.0000 - false_positives: 302932.0000 - true_positives: 106255816.0000 - true_negatives: 475565.0000 - val_loss: 3.7590 - val_binary_accuracy: 0.7474 - val_false_negatives: 12889516.0000 - val_false_positives: 114471.0000 - val_true_positives: 39121896.0000 - val_true_negatives: 40771.0000
Epoch 2/20
1001/1001 [==============================] - 575s 574ms/step - loss: 0.4404 - binary_accuracy: 0.9669 - false_negatives: 6896858.0000 - false_positives: 723323.0000 - true_positives: 254730832.0000 - true_negatives: 55174.0000 - val_loss: 0.1698 - val_binary_accuracy: 0.9807 - val_false_negatives: 628226.0000 - val_false_positives: 150348.0000 - val_true_positives: 51383196.0000 - val_true_negatives: 4894.0000
Epoch 3/20
 496/1001 [=============>................] - ETA: 4:34 - loss: 0.1711 - binary_accuracy: 0.9834 - false_negatives: 1251490.0000 

In [ ]:
model.save('./DexiNed.h5')

In [ ]:
img = read_image('./dataset/render_1001.png')
img = np.expand_dims(img, axis=0)
#plt.imshow(img)


In [ ]:
res = model.predict(img)

In [ ]:
print(res.shape)
res = res[-1,:,:]
print(res.shape)

In [ ]:
res = res[:,:,-1]
print(res.shape)

In [ ]:
plt.imshow(res)
#plt.show()

In [ ]:
img = read_edge_convert('./dataset/render_1001_edge.png')
plt.imshow(img)